# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Jesús Emmanuel Flores Cortés  |
| **Fecha**      | 18/09/2025  |
| **Expediente** | 751571  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para poder entrenar los datos por partes, y con eso poder saber si nuestro modelo funciona con los datos usados y nuevos datos

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Se podria, pero si usamos todos los datos para entrenar no podriamos saber si funciona bien el modelo

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error


In [6]:
df = pd.read_excel('Motor Trend Car Road Tests.xlsx')

In [7]:
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [8]:
df = df.drop(columns=['model'])

In [9]:
X = df.drop(columns=['mpg']).values
y = df['mpg'].values

In [10]:
loo = LeaveOneOut()
mse_scores = []

In [11]:
for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)


In [12]:
print(mse_scores)
print(f"MSE promedio: {np.mean(mse_scores):.4f}")
print(f"Desviación estándar del MSE: {np.std(mse_scores):.4f}")

[5.258855796629049, 2.453996713086807, 20.550493416882723, 0.04432910713963862, 1.5811555159760176, 10.118625488875358, 0.01637404154847087, 8.081250710389458, 39.47018347964974, 0.7706327722901085, 4.955694234215848, 10.224730198354523, 4.430129331402691, 0.48781277124784445, 6.82384457580952, 0.6027506975547007, 36.810178609958804, 31.759855581210353, 1.063227541805789, 32.71328618702726, 14.307173682712742, 3.4053546499988485, 9.408007649079861, 0.00010350039803642291, 9.96243953454772, 1.3412208919431232, 0.16480064082222126, 23.600206502249215, 83.11457643385152, 0.00010271237184925128, 8.785533367967178, 17.502929883865395]
MSE promedio: 12.1816
Desviación estándar del MSE: 17.0674


Interpreta.

Con el LOOCV pudimos ver de manera "general" como se desempeña el modelo, con las muestras que sacamos, y el promedio de error de esas muestras es algo alto, al igual que la desviacion, osea que el modelo nos es muy preciso pero nos ayuda a darnos una idea.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [24]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [26]:
housing_x = pd.DataFrame(data=X, columns=housing.feature_names)
housing_x

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [27]:
housing_y = pd.DataFrame(data=y, columns=housing.target_names)
housing_y

,MedHouseVal
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422
...,...
20635,0.781
20636,0.771
20637,0.923
20638,0.847


In [29]:
housing_full = pd.concat([housing_x, housing_y], axis=1)
housing_full

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [38]:
housing_shuffled = housing_full.sample(frac=1, random_state=137)

In [39]:
housing_shuffled

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
17500,5.1615,26.0,6.687037,1.042593,2089.0,3.868519,34.43,-119.79,2.762
18701,3.2639,22.0,5.504673,0.906542,338.0,3.158879,40.57,-122.35,0.875
1738,4.4135,32.0,5.403846,0.956044,1144.0,3.142857,37.98,-122.33,1.501
7984,3.7527,38.0,4.410596,1.036424,1627.0,2.693709,33.87,-118.18,1.619
14857,1.8672,43.0,3.988095,0.912698,548.0,2.174603,32.64,-117.08,1.458
...,...,...,...,...,...,...,...,...,...
12417,2.8648,29.0,4.740586,0.924686,918.0,3.841004,33.73,-116.23,0.721
4430,1.9704,34.0,3.397614,1.117296,2105.0,4.184891,34.08,-118.22,1.521
1171,2.1658,29.0,5.777448,1.136499,925.0,2.744807,39.50,-121.58,0.576
18726,1.6033,19.0,5.699454,1.114754,505.0,2.759563,40.58,-122.30,0.988


In [41]:
folds = [housing_shuffled.iloc[i::10, :] for i in range(10)]


In [45]:
mse_folds = []

In [49]:
for i in range(10):
    test_folds = folds[i]
    train_folds = pd.concat([folds[j] for j in range(10) if j != i])

    X_train = train_folds.drop(columns=['MedHouseVal']).values
    y_train = train_folds['MedHouseVal'].values
    X_test = test_folds.drop(columns=['MedHouseVal']).values
    y_test = test_folds['MedHouseVal'].values

    lm = LinearRegression()
    lm.fit(X_train, y_train)
    y_pred = lm.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_folds.append(mse)
    print(f"Fold {i+1}, MSE: {mse:.4f}")

print(f" MSE promedio: {np.mean(mse_folds):.4f}")
print(f"Desviación estándar del MSE: {np.std(mse_folds):.4f}")


Fold 1, MSE: 0.4775
Fold 2, MSE: 0.5239
Fold 3, MSE: 0.6003
Fold 4, MSE: 0.5380
Fold 5, MSE: 0.4878
Fold 6, MSE: 0.5488
Fold 7, MSE: 0.5210
Fold 8, MSE: 0.4847
Fold 9, MSE: 0.5629
Fold 10, MSE: 0.5424
 MSE promedio: 0.5287
Desviación estándar del MSE: 0.0364


Interpreta.

Aqui tambien divimos nuestro df en 10 partes (folds), para entrenar con 9 y probarlo con la que dejamos fuera, asi hasta porbarlo con todas. Con eso calculamos el MSE y la desviación.
Con esta manera podemos saber si el modelo generaliza de buena manera, y con los resultados que arroja nos indica que nuestro modelo es bueno porque son pequeños, a comparación del primer ejercicio que eran mayores.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3